In [1]:
import multiprocessing as mp
from pathlib import Path

import numpy as np
import pandas as pd
import wfdb
import wfdb.io
from tqdm import tqdm

----

In [2]:
master_header = wfdb.io.rdheader('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p04/p044083/p044083-2112-05-04-19-50', rd_segments=True)
master_header

In [3]:
segment_header = wfdb.io.rdheader('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p04/p044083/3343009_0001')
segment_header

In [4]:
master_header.sig_name

['II', 'AVR', 'V', 'MCL', 'RESP', 'PLETH', 'ABP']

In [5]:
master_header.contained_ranges('II')

[(0, 7535000),
 (7535658, 15344033),
 (15345033, 16154283),
 (16155283, 16363783),
 (16364658, 16633533),
 (16634283, 17151158),
 (17152033, 17368908),
 (17369783, 20342033)]

In [6]:
master_header.contained_combined_ranges(['II', 'RESP', 'PLETH', 'ABP'])

[(38875, 6203500), (6281500, 7535000), (7535658, 9734908)]

In [7]:
# total_samples = sum([end - start for start, end in master_header.contained_combined_ranges(relevant_signals)])
# total_minutes = total_samples / 125 / 60
# print(f'{total_minutes:.2f} min')

In [8]:
req_signals = ['II', 'PLETH', 'RESP']
min_time = 5 * 60
good_times = []
good_segments = []
for seg in master_header.segments:
    if seg is not None:
        seg: wfdb.io.Record
        if seg.sig_len > seg.fs * min_time:
            if all([sig in seg.sig_name for sig in req_signals]):
                good_times.append(seg.sig_len / seg.fs)
                good_segments.append(seg.record_name)
print(f'{len(good_segments)} segments, {sum(good_times)/60:.2f} minutes')

14 segments, 2684.52 minutes


In [9]:
good_segments

['3314767_0005',
 '3314767_0007',
 '3314767_0010',
 '3314767_0013',
 '3314767_0017',
 '3314767_0020',
 '3314767_0021',
 '3314767_0023',
 '3314767_0026',
 '3314767_0029',
 '3314767_0032',
 '3314767_0035',
 '3314767_0038',
 '3314767_0041']

----

In [ ]:
relevant_signals = ['II', 'PLETH', 'RESP', 'ABP']

In [11]:
mimic_root = Path('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0')

In [12]:
p_folders = sorted([f for g in mimic_root.iterdir() if g.is_dir() for f in g.iterdir() if f.is_dir()])
p_folders

[PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000020'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000030'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000033'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000052'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000079'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000085'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000107'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000109'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000123'),
 PosixPath('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p00/p000124'),
 PosixPath('/home/th

In [81]:
def process_headers(p_folder: Path):
    master_files = [file for file in p_folder.iterdir() if file.name.startswith('p') and not file.name.endswith('n.hea')]
    data = []
    for master_file in master_files:
        try:
            master_header = wfdb.io.rdheader(master_file.as_posix().replace('.hea', ''), rd_segments=True)
        except (FileNotFoundError, TypeError):
            continue
        subject_id_str, record_start_str = master_file.name.split('-', 1)
        subject_id = int(subject_id_str[1:])
        record_start = pd.to_datetime(record_start_str.replace('.hea', ''), format='%Y-%m-%d-%H-%M')
        min_time = 5 * 60
        seg_times = []
        seg_records = []
        for seg in master_header.segments:
            if seg is not None:
                seg: wfdb.io.Record
                if seg.sig_len > seg.fs * min_time:
                    if all([sig in seg.sig_name for sig in relevant_signals]):
                        seg_times.append(seg.sig_len / seg.fs)
                        seg_records.append(seg.record_name)
        if len(seg_times) > 0:
            data.append({'subject_id': subject_id, 'record_start': record_start, 'total_time': sum(seg_times)})
    return data


with mp.Pool(8) as p:
    data_mp = list(tqdm(p.imap_unordered(func=process_headers, iterable=p_folders, chunksize=1), total=len(p_folders)))
df_waveforms_mp = pd.DataFrame([y for x in data_mp for y in x]).sort_values(['subject_id', 'record_start'])
df_waveforms_mp

100%|██████████| 10282/10282 [01:59<00:00, 85.91it/s] 


,subject_id,record_start,total_time
0,160,2174-11-06 10:12:00,10789.0
3,188,2161-12-09 17:50:00,80434.0
2,188,2161-12-10 17:58:00,6216.0
1,188,2162-01-10 18:53:00,473998.0
5,543,2148-12-13 21:51:00,160737.0
...,...,...,...
3294,99863,2142-04-06 18:42:00,73459.0
3291,99883,2131-12-24 17:38:00,98832.0
3292,99883,2131-12-26 09:41:00,28912.0
3298,99894,2129-07-07 02:43:00,117572.0


----

In [86]:
df_admissions = pd.read_csv('/home/thulio/Downloads/mimic/ADMISSIONS.csv',
                            index_col='ROW_ID',
                            parse_dates=['ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'EDREGTIME', 'EDOUTTIME']).sort_index()
df_admissions

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
ROW_ID,,,,,,,,,,,,,,,,,,
1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaT,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,NOT SPECIFIED,NaN,ASIAN,NaT,NaT,NEWBORN,0,1
2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE,2101-10-20 17:09:00,2101-10-20 19:24:00,HYPOTENSION,0,1
3,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,2191-03-15 13:10:00,2191-03-16 01:10:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",0,1
4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaT,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,BUDDHIST,NaN,ASIAN,NaT,NaT,NEWBORN,0,1
5,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaT,NaT,CHRONIC RENAL FAILURE/SDA,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58972,99985,176670,2181-01-27 02:47:00,2181-02-12 17:05:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,ENGL,JEWISH,MARRIED,WHITE,2181-01-26 23:35:00,2181-01-27 04:18:00,FEVER,0,1
58973,99991,151118,2184-12-24 08:30:00,2185-01-05 12:15:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaT,NaT,DIVERTICULITIS/SDA,0,1
58974,99992,197084,2144-07-25 18:03:00,2144-07-28 17:56:00,NaT,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2144-07-25 13:40:00,2144-07-25 18:50:00,RETROPERITONEAL HEMORRHAGE,0,1


In [87]:
df_icustays = pd.read_csv('/home/thulio/Downloads/mimic/ICUSTAYS.csv',
                          index_col='ROW_ID',
                          parse_dates=['INTIME', 'OUTTIME']).sort_index()
df_icustays

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
ROW_ID,,,,,,,,,,,
1,2,163353,243653,carevue,NICU,NICU,56,56,2138-07-17 21:20:07,2138-07-17 23:32:21,0.0918
2,3,145834,211552,carevue,MICU,MICU,12,12,2101-10-20 19:10:11,2101-10-26 20:43:09,6.0646
3,4,185777,294638,carevue,MICU,MICU,52,52,2191-03-16 00:29:31,2191-03-17 16:46:31,1.6785
4,5,178980,214757,carevue,NICU,NICU,56,56,2103-02-02 06:04:24,2103-02-02 08:06:00,0.0844
5,6,107064,228232,carevue,SICU,SICU,33,33,2175-05-30 21:30:54,2175-06-03 13:39:54,3.6729
...,...,...,...,...,...,...,...,...,...,...,...
61528,99985,176670,279638,metavision,MICU,MICU,52,52,2181-01-29 05:33:34,2181-02-09 12:45:20,11.2998
61529,99991,151118,226241,metavision,TSICU,TSICU,14,14,2184-12-28 17:30:58,2184-12-31 20:56:20,3.1426
61530,99992,197084,242052,metavision,MICU,MICU,50,50,2144-07-25 18:04:42,2144-07-27 17:27:55,1.9745


In [88]:
df_diagnoses = pd.read_csv('/home/thulio/Downloads/mimic/DIAGNOSES_ICD.csv', index_col='ROW_ID').dropna().sort_index()
df_diagnoses

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
ROW_ID,,,,
1,2,163353,1.0,V3001
2,2,163353,2.0,V053
3,2,163353,3.0,V290
4,3,145834,1.0,0389
5,3,145834,2.0,78559
...,...,...,...,...
651043,99999,113369,1.0,75612
651044,99999,113369,2.0,7861
651045,99999,113369,3.0,4019


In [ ]:
# def has_interfering_diseases(df: pd.DataFrame):
#     return df['ICD9_CODE'].str.startswith(('40', '41', '42', '43')).any()

# df_diagnoses_no_inter = df_diagnoses.groupby(['SUBJECT_ID', 'HADM_ID']).filter(has_interfering_diseases)
# df_diagnoses_no_inter

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
ROW_ID,,,,
1,2,163353,1.0,V3001
2,2,163353,2.0,V053
3,2,163353,3.0,V290
13,4,185777,1.0,042
14,4,185777,2.0,1363
...,...,...,...,...
650992,99985,176670,9.0,2724
650993,99985,176670,10.0,2859
650994,99985,176670,11.0,53081


In [ ]:
def has_interfering_diseases(df: pd.DataFrame):
    return df['ICD9_CODE'].str.startswith(('40', '41', '42', '43')).any()

df_diagnoses_no_inter = df_diagnoses.groupby(['SUBJECT_ID', 'HADM_ID']).filter(has_interfering_diseases)
df_diagnoses_no_inter

In [102]:
df_diagnoses.groupby(['SUBJECT_ID', 'HADM_ID'])['ICD9_CODE'].apply(has_interfering_diseases)

SUBJECT_ID  HADM_ID
2           163353     False
3           145834      True
4           185777     False
5           178980     False
6           107064      True
                       ...  
99985       176670     False
99991       151118      True
99992       197084      True
99995       137810      True
99999       113369      True
Name: ICD9_CODE, Length: 58929, dtype: bool

In [ ]:
def has_interfering_diseases(s: pd.Series):
    return s.str.startswith(('40', '41', '42', '43')).any()

df_diagnoses_filters = df_diagnoses.copy()
df_diagnoses_filters['has_interference'] = df_diagnoses_filters.groupby(['SUBJECT_ID', 'HADM_ID'])['ICD9_CODE'].apply(has_interfering_diseases)
df_diagnoses_filters

TypeError: incompatible index of inserted column with frame index

In [ ]:
df_diagnoses_filters = df_diagnoses.copy()

df_diagnoses_filters['has_interference'] = df_diagnoses.groupby(['SUBJECT_ID', 'HADM_ID'])
df_diagnoses_filters['is_diabetic'] =
df_diagnoses_filters

In [ ]:
def is_diabetic(df: pd.DataFrame):
    # return ~df['ICD9_CODE'].str.fullmatch(r'(40|41|42|43)\d+').any()
    return ~df['ICD9_CODE'].str.startswith(('40', '41', '42', '43')).any()

df_diagnoses_diabetic = df_diagnoses.groupby(['SUBJECT_ID', 'HADM_ID']).filter(lambda df: (~df['ICD9_CODE'].str.fullmatch(r'(40|41|42|43)\d+')).all() and (df['ICD9_CODE'].str.fullmatch(r'250\d+')).any())
df_diagnoses_no_inter

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
ROW_ID,,,,
2228,184,137477,1.0,1919
2229,184,137477,2.0,6826
2230,184,137477,3.0,486
2231,184,137477,4.0,2554
2232,184,137477,5.0,25000
...,...,...,...,...
650783,99928,104802,1.0,25013
650784,99928,104802,2.0,V5867
650785,99928,104802,3.0,V1581


In [ ]:
def criterion(df):
    cardiac_or_pulmonary = df['ICD9_CODE'].str.fullmatch(r'(40|41|42|43)\d+').any()
    diabetes = df['ICD9_CODE'].str.fullmatch(r'250\d+').any()
    return diabetes and not cardiac_or_pulmonary

df_diagnoses_4 = df_diagnoses.groupby(['SUBJECT_ID', 'HADM_ID']).filter(criterion)
df_diagnoses_4 = df_diagnoses_4[df_diagnoses_4['ICD9_CODE'].str.fullmatch(r'250\d+')]
df_diagnoses_4

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
ROW_ID,,,,
2232,184,137477,5.0,25000
2348,188,164735,22.0,25000
2355,188,191517,5.0,25000
2769,224,169158,4.0,25000
3251,271,173727,5.0,25001
...,...,...,...,...
648509,99444,136391,11.0,25001
648906,99514,113538,3.0,25060
649286,99573,114347,7.0,25000


In [ ]:
df_icustays.value_counts('HADM_ID')

HADM_ID
163281    7
186706    7
175448    6
109520    6
123178    6
         ..
134329    1
134332    1
134333    1
134334    1
134313    1
Name: count, Length: 57786, dtype: int64

In [ ]:
df_merge = pd.merge_asof(df_waveforms.sort_values('record_start'),
                         df_icustays.sort_values('INTIME'),
                         left_on='record_start',
                         right_on='INTIME',
                         left_by='subject_id',
                         right_by='SUBJECT_ID',
                         tolerance=pd.to_timedelta('8 hours'),
                         direction='backward').dropna(subset='SUBJECT_ID')
df_merge

,subject_id,record_start,total_time,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,72723,2100-07-09 16:06:00,76556.000,72723.0,146616.0,267483.0,metavision,SICU,SICU,33.0,33.0,2100-07-09 13:19:40,2100-07-10 16:19:58,1.1252
1,46467,2100-09-08 15:15:00,69694.736,46467.0,185232.0,257089.0,metavision,CSRU,CSRU,12.0,12.0,2100-09-08 15:12:26,2100-09-09 10:42:54,0.8128
2,41154,2100-09-11 11:37:00,105517.000,41154.0,180242.0,260576.0,metavision,CCU,CCU,7.0,7.0,2100-09-11 11:13:29,2100-09-12 18:31:29,1.3042
4,92982,2100-09-22 17:37:00,49245.000,92982.0,138442.0,202181.0,metavision,SICU,SICU,57.0,57.0,2100-09-22 15:16:30,2100-09-23 12:14:35,0.8737
7,72091,2100-09-25 22:19:00,236559.000,72091.0,168925.0,285220.0,metavision,MICU,MICU,52.0,23.0,2100-09-25 17:04:29,2100-09-28 16:21:48,2.9704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8873,74493,2206-06-09 17:46:00,160261.000,74493.0,198110.0,297759.0,metavision,CCU,CCU,7.0,7.0,2206-06-09 16:44:16,2206-06-11 18:57:47,2.0927
8875,3987,2206-06-30 19:13:00,172651.000,3987.0,106870.0,298239.0,metavision,CCU,CCU,7.0,7.0,2206-06-30 19:11:52,2206-07-03 13:55:32,2.7803
8881,3792,2206-10-20 06:30:00,6390.000,3792.0,132278.0,291926.0,metavision,CSRU,CCU,15.0,7.0,2206-10-20 04:45:51,2206-10-20 16:59:34,0.5095
8885,19338,2208-04-27 23:18:00,123286.000,19338.0,103944.0,240541.0,metavision,MICU,MICU,23.0,23.0,2208-04-27 22:13:44,2208-04-29 10:18:55,1.5036


In [ ]:
df_final = pd.merge(df_merge, df_admissions, how='left', on=['SUBJECT_ID', 'HADM_ID'])
df_final = pd.merge(df_final, df_diagnoses_4, how='inner', on=['SUBJECT_ID', 'HADM_ID'])
df_final

,subject_id,record_start,total_time,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,...,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,SEQ_NUM,ICD9_CODE
0,6085,2104-10-04 01:37:00,135207.0,6085.0,174506.0,212627.0,carevue,MICU,MICU,50.0,...,PROTESTANT QUAKER,MARRIED,BLACK/AFRICAN AMERICAN,2104-10-03 19:49:00,2104-10-04 01:25:00,RECTAL BLEEDING,0,1,8.0,25000
1,96240,2105-05-04 02:48:00,28826.0,96240.0,168044.0,269358.0,metavision,SICU,SICU,33.0,...,NOT SPECIFIED,SINGLE,WHITE,2105-05-03 20:50:00,2105-05-04 02:00:00,SEPSIS,0,1,17.0,25000
2,43116,2107-03-31 18:15:00,10688.0,43116.0,107501.0,295612.0,metavision,SICU,SICU,57.0,...,CATHOLIC,NaN,WHITE,2107-03-31 16:15:00,2107-03-31 18:17:00,BRAIN MASS,0,1,7.0,25000
3,13564,2113-12-22 02:59:00,104823.0,13564.0,159282.0,202588.0,carevue,MICU,MICU,50.0,...,PROTESTANT QUAKER,MARRIED,BLACK/AFRICAN AMERICAN,2113-12-21 20:08:00,2113-12-22 01:00:00,HYPERGLYCEMIA-LEFT SIDED WEAKNESS,0,1,1.0,25002
4,28525,2114-07-21 04:31:00,132844.0,28525.0,139657.0,271400.0,carevue,MICU,MICU,50.0,...,CATHOLIC,MARRIED,WHITE,2114-07-20 22:50:00,2114-07-21 04:11:00,URINARY TRACT INFECTION;SEIZURE,0,1,13.0,25000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,27639,2191-05-30 01:21:00,74355.0,27639.0,112865.0,298997.0,carevue,MICU,MICU,50.0,...,NOT SPECIFIED,DIVORCED,BLACK/AFRICAN AMERICAN,2191-05-29 16:31:00,2191-05-30 01:09:00,SEPSIS,0,1,7.0,25000
77,71213,2194-01-23 21:22:00,49777.0,71213.0,189888.0,226283.0,metavision,SICU,SICU,33.0,...,JEWISH,WIDOWED,WHITE,2194-01-22 16:17:00,2194-01-22 21:03:00,LEFT HIP FRACTURE,0,1,12.0,25000
78,40724,2194-12-22 23:57:00,325781.0,40724.0,149365.0,295857.0,metavision,SICU,MICU,33.0,...,NOT SPECIFIED,MARRIED,WHITE,2194-12-22 17:17:00,2194-12-22 19:48:00,HYPOTENSION,0,1,13.0,25000
79,17954,2195-04-14 07:16:00,112727.0,17954.0,188143.0,257548.0,carevue,CCU,CCU,7.0,...,OTHER,NaN,UNKNOWN/NOT SPECIFIED,2195-04-06 06:24:00,2195-04-06 09:32:00,HIP FRACTURE;RENAL INSUFFIENCY,0,1,14.0,25000


In [ ]:
df_final['SUBJECT_ID'].drop_duplicates()

0      6085.0
1     96240.0
2     43116.0
3     13564.0
4     28525.0
       ...   
76    27639.0
77    71213.0
78    40724.0
79    17954.0
80    68172.0
Name: SUBJECT_ID, Length: 69, dtype: float64